# ML Model with LinearLearner

In [2]:
import os

import time
from time import gmtime, strftime

import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


## Import the preprocessed data

In [5]:
features = pd.read_csv('data/features_completed.csv', index_col=0)
labels = pd.read_csv('data/labels_completed.csv', index_col=0)

In [7]:
features.shape, labels.shape

((63288, 16), (63288, 1))

## Create Sagemaker items

In [8]:
import sagemaker

In [9]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()



## Create Training, Validation and Testdata
To avoid overfitting I split the train data additional in validation data.

In [10]:
from sklearn.model_selection import train_test_split 

In [12]:
# We split the dataset into 2/3 training and 1/3 testing sets.
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.33)



In [13]:
# import LinearLearner
from sagemaker import LinearLearner

# specify an output path
prefix = 'creditcard'
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate LinearLearner
linear = LinearLearner(role=role,
                       instance_count=1, 
                       instance_type='ml.c4.xlarge',
                       predictor_type='binary_classifier',
                       output_path=output_path,
                       sagemaker_session=sagemaker_session,
                       epochs=15)



In [16]:
# convert features/labels to numpy
train_x_np = features.values.astype('float32')
train_y_np = labels.values.astype('float32').flatten()

train_x_np
train_y_np

array([1., 1., 1., ..., 0., 0., 0.], dtype=float32)

In [18]:
# convert features/labels to numpy
train_x_np = features.values.astype('float32')
train_y_np = labels.values.astype('float32').flatten()


#type(train_x_np)
# create RecordSet
formatted_train_data = linear.record_set(train_x_np, labels=train_y_np)

In [19]:
%%time 
# train the estimator on formatted training data
linear.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-10-04 11:15:06 Starting - Starting the training job...
2021-10-04 11:15:19 Starting - Launching requested ML instancesProfilerReport-1633346106: InProgress
...
2021-10-04 11:16:06 Starting - Preparing the instances for training............
2021-10-04 11:18:07 Downloading - Downloading input data...
2021-10-04 11:18:31 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[10/04/2021 11:18:38 INFO 140110658807616] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_b

In [21]:
%%time 
# deploy and create a predictor
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------!CPU times: user 165 ms, sys: 4.8 ms, total: 170 ms
Wall time: 3min 32s


In [22]:
features.values.astype('float32')

array([[ 0.18072289,  0.46666667,  5.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.07228915,  0.33333334,  5.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.44578314,  0.4888889 ,  5.        , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.4384762 ,  0.3933888 , 10.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.7590361 ,  0.3888889 , 10.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.02409638,  0.15555556, 10.        , ...,  1.        ,
         0.        ,  0.        ]], dtype=float32)

### Helper function for evaluation


The provided function below, takes in a deployed predictor, some test features and labels, and returns a dictionary of metrics; calculating false negatives and positives as well as recall, precision, and accuracy.

In [23]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # We have a lot of test data, so we'll split it into batches of 100
    # split the test data set into batches and evaluate using prediction endpoint    
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 100)]
    
    # LinearLearner produces a `predicted_label` for each data point in a batch
    # get the 'predicted_label' for every point in a batch
    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch]) 
                                 for batch in prediction_batches])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}


In [25]:
labels.values.flatten()

array([1, 1, 1, ..., 0, 0, 0])

### Test Results

The cell below runs the `evaluate` function. 

The code assumes that you have a defined `predictor` and `test_features` and `test_labels` from previously-run cells.

In [28]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   features.values.astype('float32'), 
                   labels.values.flatten(), 
                   verbose=True) # verbose means we'll print out the metrics


Metrics for simple, LinearLearner.

prediction (col)    0.0    1.0
actual (row)                  
0                 12804  13330
1                  6092  31062

Recall:     0.836
Precision:  0.700
Accuracy:   0.693



In [29]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint_name)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint_name))

In [30]:
# delete the predictor endpoint 
delete_endpoint(linear_predictor)

Already deleted: linear-learner-2021-10-04-11-58-20-842


In [31]:
# instantiate a LinearLearner
# tune the model for a higher recall
linear_recall = LinearLearner(role=role,
                              instance_count=1, 
                              instance_type='ml.m4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              target_recall=0.9) # 90% recall



In [32]:
%%time 
# train the estimator on formatted training data
linear_recall.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-10-04 12:05:55 Starting - Starting the training job...
2021-10-04 12:06:18 Starting - Launching requested ML instancesProfilerReport-1633349154: InProgress
......
2021-10-04 12:07:19 Starting - Preparing the instances for training.........
2021-10-04 12:08:49 Downloading - Downloading input data...
2021-10-04 12:09:19 Training - Downloading the training image...
2021-10-04 12:09:40 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[10/04/2021 12:09:46 INFO 140633106163520] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method'

In [33]:
%%time 
# deploy and create a predictor
recall_predictor = linear_recall.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-----------!CPU times: user 298 ms, sys: 5.04 ms, total: 303 ms
Wall time: 5min 33s


In [34]:
print('Metrics for tuned (recall), LinearLearner.\n')

# get metrics for tuned predictor
metrics = evaluate(recall_predictor, 
                   features.values.astype('float32'), 
                   labels.values.flatten(), 
                   verbose=True)

Metrics for tuned (recall), LinearLearner.

prediction (col)   0.0    1.0
actual (row)                 
0                 9669  16465
1                 3559  33595

Recall:     0.904
Precision:  0.671
Accuracy:   0.684



## Delete the endpoint 

As always, when you're done evaluating a model, you should delete the endpoint. Below, I'm using the `delete_endpoint` helper function I defined earlier.

In [45]:
# delete the predictor endpoint 
delete_endpoint(recall_predictor)

Already deleted: linear-learner-2021-09-21-11-07-47-455


In [49]:
sagemaker_session.delete_endpoint(endpoint_name='linear-learner-2021-09-21-11-07-47-455')

In [50]:
# instantiate a LinearLearner

# include params for tuning for higher recall
# *and* account for class imbalance in training data
linear_balanced = LinearLearner(role=role,
                                instance_count=1, 
                                instance_type='ml.c4.xlarge',
                                predictor_type='binary_classifier',
                                output_path=output_path,
                                sagemaker_session=sagemaker_session,
                                epochs=15,
                                binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                                target_recall=0.9,
                                positive_example_weight_mult='balanced')


### EXERCISE: Train the balanced estimator

Fit the new, balanced estimator on the formatted training data.

In [51]:
%%time 
# train the estimator on formatted training data
linear_balanced.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-09-21 11:25:46 Starting - Starting the training job...
2021-09-21 11:25:48 Starting - Launching requested ML instancesProfilerReport-1632223546: InProgress
......
2021-09-21 11:27:16 Starting - Preparing the instances for training.........
2021-09-21 11:28:39 Downloading - Downloading input data...
2021-09-21 11:29:17 Training - Downloading the training image.Docker entrypoint called with argument(s): train
Running default environment configuration script
[09/21/2021 11:29:23 INFO 139736075499328] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 

### EXERCISE: Deploy and evaluate the balanced estimator

Deploy the balanced predictor and evaluate it. Do the results match with your expectations?

In [55]:
%%time 
# deploy and create a predictor
balanced_predictor = linear_balanced.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


------------!CPU times: user 281 ms, sys: 12.6 ms, total: 293 ms
Wall time: 6min 3s


In [57]:
print('Metrics for balanced, LinearLearner.\n')

# get metrics for balanced predictor
metrics = evaluate(balanced_predictor, 
                   features.values.astype('float32'), 
                   targets.values.flatten(), 
                   verbose=True)

Metrics for balanced, LinearLearner.

prediction (col)    0.0    1.0
actual (row)                  
0                 12959  20204
1                  4233  38881

Recall:     0.902
Precision:  0.658
Accuracy:   0.680



## Delete the endpoint 

When you're done evaluating a model, you should delete the endpoint.

In [60]:
sagemaker_session.delete_endpoint(endpoint_name=balanced_predictor.endpoint_name)

In [65]:
sagemaker_session.delete_endpoint(endpoint_name='linear-learner-2021-09-21-11-32-07-490')

In [61]:
# delete the predictor endpoint 
delete_endpoint(balanced_predictor)

Already deleted: linear-learner-2021-09-21-11-45-05-238
